In [82]:
import numpy as np
import re
import random

import tqdm

from elasticsearch import RequestError

import sys; sys.path.insert(0, "..") 

from ai_es_utils.queries.components import CoordinatesLocationAndCountryQuery
from ai_es_utils.services.search import GeoLocationFromCoordinatesService

from ai_es_utils.queries.models.payload import RequestPayload, Query

In [75]:
def extract_geo_point(document: dict):
    geo_point = document["_source"]["geoPoint"]
    return [geo_point["lat"], geo_point["lon"]]

In [76]:
ELASTICSEARCH_URL = "es-dev.internal.talentwunder.com"

# Set german cities

In [40]:
wikipedia_list_of_countries = """
=== A ===
{{TOC}}
{|
| colspan="3" |
|-
| width="400" valign="top" |
:[[Aach (Hegau)|Aach]] (BW)
:[[Aachen]] (NW)
:[[Aalen]] (BW)
:[[Abenberg]] (BY)
:[[Abensberg]] (BY)
:[[Achern]] (BW)
:[[Achim (Landkreis Verden)|Achim]] (NI)
:[[Adelsheim]] (BW)
:[[Adenau]] (RP)
:[[Adorf/Vogtl.]] (SN)
:[[Ahaus]] (NW)
:[[Ahlen]] (NW)
:[[Ahrensburg]] (SH)
:[[Aichach]] (BY)
:[[Aichtal]] (BW)
:[[Aken (Elbe)]] (ST)
:[[Albstadt]] (BW)
:[[Alfeld (Leine)]] (NI)
:[[Allendorf (Lumda)]] (HE)
:[[Allstedt]] (ST)
:[[Alpirsbach]] (BW)
:[[Alsdorf]] (NW)
:[[Alsfeld]] (HE)
:[[Alsleben (Saale)]] (ST)
| width="400" valign="top" |
:[[Altdorf bei Nürnberg]] (BY)
:[[Altena]] (NW)
:[[Altenberg (Erzgebirge)|Altenberg]] (SN)
:[[Altenburg]] (TH)
:[[Altenkirchen (Westerwald)]] (RP)
:[[Altensteig]] (BW)
:[[Altentreptow]] (MV)
:[[Altlandsberg]] (BB)
:[[Altötting]] (BY)
:[[Alzenau]] (BY)
:[[Alzey]] (RP)
:[[Amberg]] (BY)
:[[Amöneburg]] (HE)
:[[Amorbach]] (BY)
:[[Amt Creuzburg]] (TH)
:[[An der Schmücke]] (TH)
:[[Andernach]] (RP)
:[[Angermünde]] (BB)
:[[Anklam]], Hansestadt (MV)
:[[Annaberg-Buchholz]] (SN)
:[[Annaburg]] (ST)
:[[Annweiler am Trifels]] (RP)
:[[Ansbach]] (BY)
:[[Apolda]] (TH)
| width="400" valign="top" |
:[[Arendsee (Altmark)]] (ST)
:[[Arneburg]] (ST)
:[[Arnis]] (SH)
:[[Arnsberg]] (NW)
:[[Arnstadt]] (TH)
:[[Arnstein (Unterfranken)|Arnstein]] (BY)
:[[Arnstein (Sachsen-Anhalt)|Arnstein]] (ST)
:[[Artern]] (TH)
:[[Arzberg (Oberfranken)|Arzberg]] (BY)
:[[Aschaffenburg]] (BY)
:[[Aschersleben]] (ST)
:[[Asperg]] (BW)
:[[Aßlar]] (HE)
:[[Attendorn]] (NW)
:[[Aub]] (BY)
:[[Aue-Bad Schlema]] (SN)
:[[Auerbach in der Oberpfalz]] (BY)
:[[Auerbach/Vogtl.]] (SN)
:[[Augsburg]] (BY)
:[[Augustusburg]] (SN)
:[[Aulendorf]] (BW)
:[[Auma-Weidatal]] (TH)
:[[Aurich]] (NI)
|}

=== B ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Babenhausen (Hessen)|Babenhausen]] (HE)
:[[Bacharach]] (RP)
:[[Backnang]] (BW)
:[[Bad Aibling]] (BY)
:[[Bad Arolsen]] (HE)
:[[Bad Belzig]] (BB)
:[[Bad Bentheim]] (NI)
:[[Bad Bergzabern]] (RP)
:[[Bad Berka]] (TH)
:[[Bad Berleburg]] (NW)
:[[Bad Berneck im Fichtelgebirge]] (BY)
:[[Bad Bevensen]] (NI)
:[[Bad Bibra]] (ST)
:[[Bad Blankenburg]] (TH)
:[[Bad Bramstedt]] (SH)
:[[Bad Breisig]] (RP)
:[[Bad Brückenau]] (BY)
:[[Bad Buchau]] (BW)
:[[Bad Camberg]] (HE)
:[[Bad Doberan]] (MV)
:[[Bad Driburg]] (NW)
:[[Bad Düben]] (SN)
:[[Bad Dürkheim]] (RP)
:[[Bad Dürrenberg]] (ST)
:[[Bad Dürrheim]] (BW)
:[[Bad Elster]] (SN)
:[[Bad Ems]] (RP)
:[[Baden-Baden]] (BW)
:[[Bad Fallingbostel]] (NI)
:[[Bad Frankenhausen/Kyffhäuser]] (TH)
:[[Bad Freienwalde (Oder)]] (BB)
:[[Bad Friedrichshall]] (BW)
:[[Bad Gandersheim]] (NI)
:[[Bad Gottleuba-Berggießhübel]] (SN)
:[[Bad Griesbach im Rottal]] (BY)
:[[Bad Harzburg]] (NI)
:[[Bad Herrenalb]] (BW)
:[[Bad Hersfeld]] (HE)
:[[Bad Homburg vor der Höhe]] (HE)
:[[Bad Honnef]] (NW)
:[[Bad Hönningen]] (RP)
:[[Bad Iburg]] (NI)
:[[Bad Karlshafen]] (HE)
:[[Bad Kissingen]] (BY)
:[[Bad König]] (HE)
:[[Bad Königshofen im Grabfeld]] (BY)
:[[Bad Köstritz]] (TH)
:[[Bad Kötzting]] (BY)
:[[Bad Kreuznach]] (RP)
:[[Bad Krozingen]] (BW)
:[[Bad Laasphe]] (NW)
:[[Bad Langensalza]] (TH)
:[[Bad Lauchstädt]] (ST)
:[[Bad Lausick]] (SN)
:[[Bad Lauterberg im Harz]] (NI)
:[[Bad Liebenstein (Gemeinde)|Bad Liebenstein]] (TH)
:[[Bad Liebenwerda]] (BB)
:[[Bad Liebenzell]] (BW)
:[[Bad Lippspringe]] (NW)
:[[Bad Lobenstein]] (TH)
:[[Bad Marienberg (Westerwald)]] (RP)
:[[Bad Mergentheim]] (BW)
:[[Bad Münder am Deister]] (NI)
:[[Bad Münstereifel]] (NW)
:[[Bad Muskau]] (SN)
:[[Bad Nauheim]] (HE)
:[[Bad Nenndorf]] (NI)
:[[Bad Neuenahr-Ahrweiler]] (RP)
:[[Bad Neustadt an der Saale]] (BY)
:[[Bad Oeynhausen]] (NW)
:[[Bad Oldesloe]] (SH)
:[[Bad Orb]] (HE)
:[[Bad Pyrmont]] (NI)
:[[Bad Rappenau]] (BW)
:[[Bad Reichenhall]] (BY)
:[[Bad Rodach]] (BY)
:[[Bad Sachsa]] (NI)
:[[Bad Säckingen]] (BW)
:[[Bad Salzdetfurth]] (NI)
:[[Bad Salzuflen]] (NW)
:[[Bad Salzungen]] (TH)
:[[Bad Saulgau]] (BW)
:[[Bad Schandau]] (SN)
:[[Bad Schmiedeberg]] (ST)
:[[Bad Schussenried]] (BW)
:[[Bad Schwalbach]] (HE)
:[[Bad Schwartau]] (SH)
:[[Bad Segeberg]] (SH)
| width="400" valign="top" |
:[[Bad Sobernheim]] (RP)
:[[Bad Soden am Taunus]] (HE)
:[[Bad Soden-Salmünster]] (HE)
:[[Bad Sooden-Allendorf]] (HE)
:[[Bad Staffelstein]] (BY)
:[[Bad Sulza]] (TH)
:[[Bad Sülze]] (MV)
:[[Bad Teinach-Zavelstein]] (BW)
:[[Bad Tennstedt]] (TH)
:[[Bad Tölz]] (BY)
:[[Bad Urach]] (BW)
:[[Bad Vilbel]] (HE)
:[[Bad Waldsee]] (BW)
:[[Bad Wildbad]] (BW)
:[[Bad Wildungen]] (HE)
:[[Bad Wilsnack]] (BB)
:[[Bad Wimpfen]] (BW)
:[[Bad Windsheim]] (BY)
:[[Bad Wörishofen]] (BY)
:[[Bad Wünnenberg]] (NW)
:[[Bad Wurzach]] (BW)
:[[Baesweiler]] (NW)
:[[Baiersdorf]] (BY)
:[[Balingen]] (BW)
:[[Ballenstedt]] (ST)
:[[Balve]] (NW)
:[[Bamberg]] (BY)
:[[Barby]] (ST)
:[[Bargteheide]] (SH)
:[[Barmstedt]] (SH)
:[[Bärnau]] (BY)
:[[Barntrup]] (NW)
:[[Barsinghausen]] (NI)
:[[Barth]] (MV)
:[[Baruth/Mark]] (BB)
:[[Bassum]] (NI)
:[[Battenberg (Eder)]] (HE)
:[[Baumholder]] (RP)
:[[Baunach]] (BY)
:[[Baunatal]] (HE)
:[[Bautzen]] (SN)
:[[Bayreuth]] (BY)
:[[Bebra]] (HE)
:[[Beckum]] (NW)
:[[Bedburg]] (NW)
:[[Beelitz]] (BB)
:[[Beeskow]] (BB)
:[[Beilngries]] (BY)
:[[Beilstein (Württemberg)|Beilstein]] (BW)
:[[Belgern-Schildau]] (SN)
:[[Bendorf]] (RP)
:[[Bensheim]] (HE)
:[[Berching]] (BY)
:[[Berga/Elster]] (TH)
:[[Bergen (Landkreis Celle)|Bergen]] (NI)
:[[Bergen auf Rügen]] (MV)
:[[Bergheim]] (NW)
:[[Bergisch Gladbach]] (NW)
:[[Bergkamen]] (NW)
:[[Bergneustadt]] (NW)
:[[Berlin]] (BE, Bundeshauptstadt)
:[[Bernau bei Berlin]] (BB)
:[[Bernburg (Saale)]] (ST)
:[[Bernkastel-Kues]] (RP)
:[[Bernsdorf (Oberlausitz)|Bernsdorf]] (SN)
:[[Bernstadt a. d. Eigen]] (SN)
:[[Bersenbrück]] (NI)
:[[Besigheim]] (BW)
:[[Betzdorf]] (RP)
:[[Betzenstein]] (BY)
:[[Beverungen]] (NW)
:[[Bexbach]] (SL)
:[[Biberach an der Riß]] (BW)
:[[Biedenkopf]] (HE)
:[[Bielefeld]] (NW)
:[[Biesenthal]] (BB)
:[[Bietigheim-Bissingen]] (BW)
:[[Billerbeck]] (NW)
:[[Bingen am Rhein]] (RP)
:[[Birkenfeld (Nahe)|Birkenfeld]] (RP)
:[[Bischofsheim in der Rhön]] (BY)
:[[Bischofswerda]] (SN)
:[[Bismark (Altmark)]] (ST)
:[[Bitburg]] (RP)
:[[Bitterfeld-Wolfen]] (ST)
:[[Blankenburg (Harz)]] (ST)
:[[Blankenhain]] (TH)
:[[Blaubeuren]] (BW)
| width="400" valign="top" |
:[[Blaustein]] (BW)
:[[Bleckede]] (NI)
:[[Bleicherode]] (TH)
:[[Blieskastel]] (SL)
:[[Blomberg]] (NW)
:[[Blumberg]] (BW)
:[[Bobingen]] (BY)
:[[Böblingen]] (BW)
:[[Bocholt]] (NW)
:[[Bochum]] (NW)
:[[Bockenem]] (NI)
:[[Bodenwerder]] (NI)
:[[Bogen (Stadt)|Bogen]] (BY)
:[[Böhlen (Sachsen)|Böhlen]] (SN)
:[[Boizenburg/Elbe]] (MV)
:[[Bonn]] (NW, Bundesstadt)
:[[Bonndorf im Schwarzwald]] (BW)
:[[Bönnigheim]] (BW)
:[[Bopfingen]] (BW)
:[[Boppard]] (RP)
:[[Borgentreich]] (NW)
:[[Borgholzhausen]] (NW)
:[[Borken]] (NW)
:[[Borken (Hessen)]] (HE)
:[[Borkum]] (NI)
:[[Borna]] (SN)
:[[Bornheim (Rheinland)|Bornheim]] (NW)
:[[Bottrop]] (NW)
:[[Boxberg (Baden)|Boxberg]] (BW)
:[[Brackenheim]] (BW)
:[[Brake (Unterweser)]] (NI)
:[[Brakel]] (NW)
:[[Bramsche]] (NI)
:[[Brandenburg an der Havel]] (BB)
:[[Brand-Erbisdorf]] (SN)
:[[Brandis]] (SN)
:[[Braubach]] (RP)
:[[Braunfels]] (HE)
:[[Braunlage]] (NI)
:[[Bräunlingen]] (BW)
:[[Braunsbedra]] (ST)
:[[Braunschweig]] (NI)
:[[Breckerfeld]] (NW)
:[[Bredstedt]] (SH)
:[[Breisach am Rhein]] (BW)
:[[Bremen]], Stadtgemeinde (HB)
:[[Bremerhaven]], Stadtgemeinde (HB)
:[[Bremervörde]] (NI)
:[[Bretten]] (BW)
:[[Breuberg]] (HE)
:[[Brilon]] (NW)
:[[Brotterode-Trusetal]] (TH)
:[[Bruchköbel]] (HE)
:[[Bruchsal]] (BW)
:[[Brück]] (BB)
:[[Brüel]] (MV)
:[[Brühl (Rheinland)|Brühl]] (NW)
:[[Brunsbüttel]] (SH)
:[[Brüssow]] (BB)
:[[Buchen (Odenwald)]] (BW)
:[[Buchholz in der Nordheide]] (NI)
:[[Buchloe]] (BY)
:[[Bückeburg]] (NI)
:[[Buckow (Märkische Schweiz)]] (BB)
:[[Büdelsdorf]] (SH)
:[[Büdingen]] (HE)
:[[Bühl (Baden)|Bühl]] (BW)
:[[Bünde]] (NW)
:[[Büren (Westfalen)|Büren]] (NW)
:[[Burg (bei Magdeburg)|Burg]] (ST)
:[[Burgau]] (BY)
:[[Burgbernheim]] (BY)
:[[Burgdorf (Region Hannover)|Burgdorf]] (NI)
:[[Bürgel (Thüringen)|Bürgel]] (TH)
:[[Burghausen]] (BY)
:[[Burgkunstadt]] (BY)
:[[Burglengenfeld]] (BY)
:[[Burgstädt]] (SN)
:[[Burg Stargard]] (MV)
:[[Burgwedel]] (NI)
:[[Burladingen]] (BW)
:[[Burscheid]] (NW)
:[[Bürstadt]] (HE)
:[[Butzbach]] (HE)
:[[Bützow]] (MV)
:[[Buxtehude]] (NI)
|}

=== C ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Calau]] (BB)
:[[Calbe (Saale)]] (ST)
:[[Calw]] (BW)
:[[Castrop-Rauxel]] (NW)
:[[Celle]] (NI)
:[[Cham (Oberpfalz)|Cham]] (BY)
:[[Chemnitz]] (SN)
:[[Clausthal-Zellerfeld]] (NI)
| width="400" valign="top" |
:[[Clingen]] (TH)
:[[Cloppenburg]] (NI)
:[[Coburg]] (BY)
:[[Cochem]] (RP)
:[[Coesfeld]] (NW)
:[[Colditz]] (SN)
:[[Coswig (Sachsen)|Coswig]] (SN)
:[[Coswig (Anhalt)]] (ST)
| width="400" valign="top" |
:[[Cottbus]] (BB)
:[[Crailsheim]] (BW)
:[[Creglingen]] (BW)
:[[Creußen]] (BY)
:[[Crimmitschau]] (SN)
:[[Crivitz]] (MV)
:[[Cuxhaven]] (NI)
|}

=== D ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Daaden]] (RP)
:[[Dachau]] (BY)
:[[Dahlen (Sachsen)|Dahlen]] (SN)
:[[Dahme/Mark]] (BB)
:[[Dahn]] (RP)
:[[Damme (Dümmer)|Damme]] (NI)
:[[Dannenberg (Elbe)]] (NI)
:[[Dargun]] (MV)
:[[Darmstadt]] (HE)
:[[Dassel]] (NI)
:[[Dassow]] (MV)
:[[Datteln]] (NW)
:[[Daun]] (RP)
:[[Deggendorf]] (BY)
:[[Deidesheim]] (RP)
:[[Delbrück]] (NW)
:[[Delitzsch]] (SN)
:[[Delmenhorst]] (NI)
:[[Demmin]], Hansestadt (MV)
:[[Dessau-Roßlau]] (ST)
:[[Detmold]] (NW)
:[[Dettelbach]] (BY)
:[[Dieburg]] (HE)
| width="400" valign="top" |
:[[Diemelstadt]] (HE)
:[[Diepholz]] (NI)
:[[Dierdorf]] (RP)
:[[Dietenheim]] (BW)
:[[Dietfurt an der Altmühl]] (BY)
:[[Dietzenbach]] (HE)
:[[Diez]] (RP)
:[[Dillenburg]] (HE)
:[[Dillingen an der Donau]] (BY)
:[[Dillingen/Saar]] (SL)
:[[Dingelstädt]] (TH)
:[[Dingolfing]] (BY)
:[[Dinkelsbühl]] (BY)
:[[Dinklage]] (NI)
:[[Dinslaken]] (NW)
:[[Dippoldiswalde]] (SN)
:[[Dissen am Teutoburger Wald]] (NI)
:[[Ditzingen]] (BW)
:[[Döbeln]] (SN)
:[[Doberlug-Kirchhain]] (BB)
:[[Döbern]] (BB)
:[[Dohna]] (SN)
:[[Dömitz]] (MV)
| width="400" valign="top" |
:[[Dommitzsch]] (SN)
:[[Donaueschingen]] (BW)
:[[Donauwörth]] (BY)
:[[Donzdorf]] (BW)
:[[Dorfen]] (BY)
:[[Dormagen]] (NW)
:[[Dornburg-Camburg]] (TH)
:[[Dornhan]] (BW)
:[[Dornstetten]] (BW)
:[[Dorsten]] (NW)
:[[Dortmund]] (NW)
:[[Dransfeld]] (NI)
:[[Drebkau]] (BB)
:[[Dreieich]] (HE)
:[[Drensteinfurt]] (NW)
:[[Dresden]] (SN, Landeshauptstadt)
:[[Drolshagen]] (NW)
:[[Duderstadt]] (NI)
:[[Duisburg]] (NW)
:[[Dülmen]] (NW)
:[[Düren]] (NW)
:[[Düsseldorf]] (NW, Landeshauptstadt)
|}

=== E ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Ebeleben]] (TH)
:[[Eberbach]] (BW)
:[[Ebermannstadt]] (BY)
:[[Ebern]] (BY)
:[[Ebersbach an der Fils]] (BW)
:[[Ebersbach-Neugersdorf]] (SN)
:[[Ebersberg]] (BY)
:[[Eberswalde]] (BB)
:[[Eckartsberga]] (ST)
:[[Eckernförde]] (SH)
:[[Edenkoben]] (RP)
:[[Egeln]] (ST)
:[[Eggenfelden]] (BY)
:[[Eggesin]] (MV)
:[[Ehingen (Donau)]] (BW)
:[[Ehrenfriedersdorf]] (SN)
:[[Eibelstadt]] (BY)
:[[Eibenstock]] (SN)
:[[Eichstätt]] (BY)
:[[Eilenburg]] (SN)
:[[Einbeck]] (NI)
:[[Eisenach]] (TH)
:[[Eisenberg (Thüringen)|Eisenberg]] (TH)
:[[Eisenberg (Pfalz)]] (RP)
:[[Eisenhüttenstadt]] (BB)
:[[Eisfeld]] (TH)
:[[Lutherstadt Eisleben|Eisleben]], Lutherstadt (ST)
| width="400" valign="top" |
:[[Eislingen/Fils]] (BW)
:[[Ellingen]] (BY)
:[[Ellrich]] (TH)
:[[Ellwangen (Jagst)]] (BW)
:[[Elmshorn]] (SH)
:[[Elsdorf (Rheinland)|Elsdorf]] (NW)
:[[Elsfleth]] (NI)
:[[Elsterberg]] (SN)
:[[Elsterwerda]] (BB)
:[[Elstra]] (SN)
:[[Elterlein]] (SN)
:[[Eltmann]] (BY)
:[[Eltville am Rhein]] (HE)
:[[Elzach]] (BW)
:[[Elze]] (NI)
:[[Emden]] (NI)
:[[Emmelshausen]] (RP)
:[[Emmendingen]] (BW)
:[[Emmerich am Rhein]] (NW)
:[[Emsdetten]] (NW)
:[[Endingen am Kaiserstuhl]] (BW)
:[[Engen]] (BW)
:[[Enger]] (NW)
:[[Ennepetal]] (NW)
:[[Ennigerloh]] (NW)
:[[Eppelheim]] (BW)
:[[Eppingen]] (BW)
| width="400" valign="top" |
:[[Eppstein]] (HE)
:[[Erbach (Donau)|Erbach]] (BW)
:[[Erbach (Odenwald)]] (HE)
:[[Erbendorf]] (BY)
:[[Erding]] (BY)
:[[Erftstadt]] (NW)
:[[Erfurt]] (TH, Landeshauptstadt)
:[[Erkelenz]] (NW)
:[[Erkner]] (BB)
:[[Erkrath]] (NW)
:[[Erlangen]] (BY)
:[[Erlenbach am Main]] (BY)
:[[Erlensee]] (HE)
:[[Erwitte]] (NW)
:[[Eschborn]] (HE)
:[[Eschenbach in der Oberpfalz]] (BY)
:[[Eschershausen]] (NI)
:[[Eschwege]] (HE)
:[[Eschweiler]] (NW)
:[[Esens]] (NI)
:[[Espelkamp]] (NW)
:[[Essen]] (NW)
:[[Esslingen am Neckar]] (BW)
:[[Ettenheim]] (BW)
:[[Ettlingen]] (BW)
:[[Euskirchen]] (NW)
:[[Eutin]] (SH)
|}

=== F ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Falkenberg/Elster]] (BB)
:[[Falkensee]] (BB)
:[[Falkenstein/Harz]] (ST)
:[[Falkenstein/Vogtl.]] (SN)
:[[Fehmarn]] (SH)
:[[Fellbach]] (BW)
:[[Felsberg (Hessen)|Felsberg]] (HE)
:[[Feuchtwangen]] (BY)
:[[Filderstadt]] (BW)
:[[Finsterwalde]] (BB)
:[[Fladungen]] (BY)
:[[Flensburg]] (SH)
:[[Flöha]] (SN)
:[[Flörsheim am Main]] (HE)
:[[Florstadt]] (HE)
:[[Forchheim]] (BY)
:[[Forchtenberg]] (BW)
:[[Forst (Lausitz)]] (BB)
:[[Frankenau]] (HE)
:[[Frankenberg (Eder)]] (HE)
:[[Frankenberg/Sa.]] (SN)
:[[Frankenthal (Pfalz)]] (RP)
| width="400" valign="top" |
:[[Frankfurt am Main]] (HE)
:[[Frankfurt (Oder)]] (BB)
:[[Franzburg]] (MV)
:[[Frauenstein (Erzgebirge)|Frauenstein]] (SN)
:[[Frechen]] (NW)
:[[Freiberg am Neckar]] (BW)
:[[Freiberg]] (SN)
:[[Freiburg im Breisgau]] (BW)
:[[Freilassing]] (BY)
:[[Freinsheim]] (RP)
:[[Freising]] (BY)
:[[Freital]] (SN)
:[[Freren]] (NI)
:[[Freudenberg (Baden)|Freudenberg]] (BW)
:[[Freudenberg (Siegerland)|Freudenberg]] (NW)
:[[Freudenstadt]] (BW)
:[[Freyburg (Unstrut)]] (ST)
:[[Freystadt]] (BY)
:[[Freyung]] (BY)
:[[Fridingen an der Donau]] (BW)
:[[Friedberg (Bayern)|Friedberg]] (BY)
:[[Friedberg (Hessen)]] (HE)
| width="400" valign="top" |
:[[Friedland (Niederlausitz)|Friedland]] (BB)
:[[Friedland (Mecklenburg)|Friedland]] (MV)
:[[Friedrichroda]] (TH)
:[[Friedrichsdorf]] (HE)
:[[Friedrichshafen]] (BW)
:[[Friedrichstadt]] (SH)
:[[Friedrichsthal (Saar)|Friedrichsthal]] (SL)
:[[Friesack]] (BB)
:[[Friesoythe]] (NI)
:[[Fritzlar]] (HE)
:[[Frohburg]] (SN)
:[[Fröndenberg/Ruhr]] (NW)
:[[Fulda]] (HE)
:[[Fürstenau]] (NI)
:[[Fürstenberg/Havel]] (BB)
:[[Fürstenfeldbruck]] (BY)
:[[Fürstenwalde/Spree]] (BB)
:[[Fürth]] (BY)
:[[Furth im Wald]] (BY)
:[[Furtwangen im Schwarzwald]] (BW)
:[[Füssen]] (BY)
|}

=== G ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Gadebusch]] (MV)
:[[Gaggenau]] (BW)
:[[Gaildorf]] (BW)
:[[Gammertingen]] (BW)
:[[Garbsen]] (NI)
:[[Garching bei München]] (BY)
:[[Gardelegen]] (ST)
:[[Garding]] (SH)
:[[Gartz (Oder)]] (BB)
:[[Garz/Rügen]] (MV)
:[[Gau-Algesheim]] (RP)
:[[Gebesee]] (TH)
:[[Gedern]] (HE)
:[[Geesthacht]] (SH)
:[[Geestland]] (NI)
:[[Gefell]] (TH)
:[[Gefrees]] (BY)
:[[Gehrden]] (NI)
:[[Geilenkirchen]] (NW)
:[[Geisa]] (TH)
:[[Geiselhöring]] (BY)
:[[Geisenfeld]] (BY)
:[[Geisenheim]] (HE)
:[[Geisingen]] (BW)
:[[Geislingen (Zollernalbkreis)|Geislingen]] (BW)
:[[Geislingen an der Steige]] (BW)
:[[Geithain]] (SN)
:[[Geldern]] (NW)
:[[Gelnhausen]] (HE)
:[[Gelsenkirchen]] (NW)
:[[Gemünden am Main]] (BY)
:[[Gemünden (Wohra)]] (HE)
:[[Gengenbach]] (BW)
:[[Genthin]] (ST)
:[[Georgsmarienhütte]] (NI)
:[[Gera]] (TH)
:[[Gerabronn]] (BW)
:[[Gerbstedt]] (ST)
:[[Geretsried]] (BY)
:[[Geringswalde]] (SN)
:[[Gerlingen]] (BW)
:[[Germering]] (BY)
:[[Germersheim]] (RP)
| width="400" valign="top" |
:[[Gernsbach]] (BW)
:[[Gernsheim]] (HE)
:[[Gerolstein]] (RP)
:[[Gerolzhofen]] (BY)
:[[Gersfeld (Rhön)]] (HE)
:[[Gersthofen]] (BY)
:[[Gescher]] (NW)
:[[Geseke]] (NW)
:[[Gevelsberg]] (NW)
:[[Geyer]] (SN)
:[[Giengen an der Brenz]] (BW)
:[[Gießen]] (HE)
:[[Gifhorn]] (NI)
:[[Ginsheim-Gustavsburg]] (HE)
:[[Gladbeck]] (NW)
:[[Gladenbach]] (HE)
:[[Glashütte (Sachsen)|Glashütte]] (SN)
:[[Glauchau]] (SN)
:[[Glinde]] (SH)
:[[Glücksburg (Ostsee)]] (SH)
:[[Glückstadt]] (SH)
:[[Gnoien]] (MV)
:[[Goch]] (NW)
:[[Goldberg]] (MV)
:[[Goldkronach]] (BY)
:[[Golßen]] (BB)
:[[Gommern]] (ST)
:[[Göppingen]] (BW)
:[[Görlitz]] (SN)
:[[Goslar]] (NI)
:[[Gößnitz (Thüringen)|Gößnitz]] (TH)
:[[Gotha]] (TH)
:[[Göttingen]] (NI)
:[[Grabow (Elde)|Grabow]] (MV)
:[[Grafenau (Niederbayern)|Grafenau]] (BY)
:[[Gräfenberg]] (BY)
:[[Gräfenhainichen]] (ST)
:[[Gräfenthal]] (TH)
:[[Grafenwöhr]] (BY)
:[[Grafing bei München]] (BY)
:[[Gransee]] (BB)
:[[Grebenau]] (HE)
| width="400" valign="top" |
:[[Grebenstein]] (HE)
:[[Greding]] (BY)
:[[Greifswald]], Hansestadt (MV)
:[[Greiz]] (TH)
:[[Greußen]] (TH)
:[[Greven]] (NW)
:[[Grevenbroich]] (NW)
:[[Grevesmühlen]] (MV)
:[[Griesheim]] (HE)
:[[Grimma]] (SN)
:[[Grimmen]] (MV)
:[[Gröditz]] (SN)
:[[Groitzsch]] (SN)
:[[Gronau (Leine)]] (NI)
:[[Gronau (Westf.)]] (NW)
:[[Gröningen]] (ST)
:[[Großalmerode]] (HE)
:[[Groß-Bieberau]] (HE)
:[[Großbottwar]] (BW)
:[[Großbreitenbach]] (TH)
:[[Großenhain]] (SN)
:[[Groß-Gerau]] (HE)
:[[Großräschen]] (BB)
:[[Großröhrsdorf]] (SN)
:[[Großschirma]] (SN)
:[[Groß-Umstadt]] (HE)
:[[Grünberg (Hessen)|Grünberg]] (HE)
:[[Grünhain-Beierfeld]] (SN)
:[[Grünsfeld]] (BW)
:[[Grünstadt]] (RP)
:[[Guben]] (BB)
:[[Gudensberg]] (HE)
:[[Güglingen]] (BW)
:[[Gummersbach]] (NW)
:[[Gundelfingen an der Donau]] (BY)
:[[Gundelsheim (Württemberg)|Gundelsheim]] (BW)
:[[Günzburg]] (BY)
:[[Gunzenhausen]] (BY)
:[[Güsten]] (ST)
:[[Güstrow]] (MV)
:[[Gütersloh]] (NW)
:[[Gützkow]] (MV)
|}

=== H ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Haan]] (NW)
:[[Hachenburg]] (RP)
:[[Hadamar]] (HE)
:[[Hagen]] (NW)
:[[Hagenbach]] (RP)
:[[Hagenow]] (MV)
:[[Haiger]] (HE)
:[[Haigerloch]] (BW)
:[[Hainichen]] (SN)
:[[Haiterbach]] (BW)
:[[Halberstadt]] (ST)
:[[Haldensleben]] (ST)
:[[Halle (Saale)]] (ST)
:[[Halle (Westf.)]] (NW)
:[[Hallenberg]] (NW)
:[[Hallstadt]] (BY)
:[[Haltern am See]] (NW)
:[[Halver]] (NW)
:[[Hamburg]], Freie und Hansestadt (HH)
:[[Hameln]] (NI)
:[[Hamm]] (NW)
:[[Hammelburg]] (BY)
:[[Hamminkeln]] (NW)
:[[Hanau]] (HE)
:[[Hannover]] (NI, Landeshauptstadt)
:[[Hann. Münden]] (NI)
:[[Harburg (Schwaben)]] (BY)
:[[Hardegsen]] (NI)
:[[Haren (Ems)]] (NI)
:[[Harsewinkel]] (NW)
:[[Hartenstein (Sachsen)|Hartenstein]] (SN)
:[[Hartha]] (SN)
:[[Harzgerode]] (ST)
:[[Haselünne]] (NI)
:[[Haslach im Kinzigtal]] (BW)
:[[Haßfurt]] (BY)
:[[Hattersheim am Main]] (HE)
:[[Hattingen]] (NW)
:[[Hatzfeld (Eder)]] (HE)
:[[Hausach]] (BW)
:[[Hauzenberg]] (BY)
:[[Havelberg]] (ST)
:[[Havelsee]] (BB)
:[[Hayingen]] (BW)
:[[Hechingen]] (BW)
:[[Hecklingen]] (ST)
:[[Heide (Holstein)|Heide]] (SH)
:[[Heideck]] (BY)
:[[Heidelberg]] (BW)
| width="400" valign="top" |
:[[Heidenau (Sachsen)|Heidenau]] (SN)
:[[Heidenheim an der Brenz]] (BW)
:[[Heilbad Heiligenstadt]] (TH)
:[[Heilbronn]] (BW)
:[[Heiligenhafen]] (SH)
:[[Heiligenhaus]] (NW)
:[[Heilsbronn]] (BY)
:[[Heimbach (Eifel)|Heimbach]] (NW)
:[[Heimsheim]] (BW)
:[[Heinsberg]] (NW)
:[[Heitersheim]] (BW)
:[[Heldburg]] (TH)
:[[Helmbrechts]] (BY)
:[[Helmstedt]] (NI)
:[[Hemau]] (BY)
:[[Hemer]] (NW)
:[[Hemmingen (Niedersachsen)|Hemmingen]] (NI)
:[[Hemmoor]] (NI)
:[[Hemsbach]] (BW)
:[[Hennef (Sieg)]] (NW)
:[[Hennigsdorf]] (BB)
:[[Heppenheim (Bergstraße)]] (HE)
:[[Herbolzheim]] (BW)
:[[Herborn]] (HE)
:[[Herbrechtingen]] (BW)
:[[Herbstein]] (HE)
:[[Herdecke]] (NW)
:[[Herdorf]] (RP)
:[[Herford]] (NW)
:[[Heringen/Helme]] (TH)
:[[Heringen (Werra)]] (HE)
:[[Hermeskeil]] (RP)
:[[Hermsdorf (Thüringen)|Hermsdorf]] (TH)
:[[Herne]] (NW)
:[[Herrenberg]] (BW)
:[[Herrieden]] (BY)
:[[Herrnhut]] (SN)
:[[Hersbruck]] (BY)
:[[Herten]] (NW)
:[[Herzberg am Harz]] (NI)
:[[Herzberg (Elster)]] (BB)
:[[Herzogenaurach]] (BY)
:[[Herzogenrath]] (NW)
:[[Hessisch Lichtenau]] (HE)
:[[Hessisch Oldendorf]] (NI)
:[[Hettingen]] (BW)
:[[Hettstedt]] (ST)
:[[Heubach]] (BW)
| width="400" valign="top" |
:[[Heusenstamm]] (HE)
:[[Hilchenbach]] (NW)
:[[Hildburghausen]] (TH)
:[[Hilden]] (NW)
:[[Hildesheim]] (NI)
:[[Hillesheim (Eifel)|Hillesheim]] (RP)
:[[Hilpoltstein]] (BY)
:[[Hirschau]] (BY)
:[[Hirschberg (Saale)|Hirschberg]] (TH)
:[[Hirschhorn (Neckar)]] (HE)
:[[Hitzacker (Elbe)]] (NI)
:[[Hochheim am Main]] (HE)
:[[Höchstadt an der Aisch]] (BY)
:[[Höchstädt an der Donau]] (BY)
:[[Hockenheim]] (BW)
:[[Hof (Saale)|Hof]] (BY)
:[[Hofgeismar]] (HE)
:[[Hofheim am Taunus]] (HE)
:[[Hofheim in Unterfranken]] (BY)
:[[Hohenberg an der Eger]] (BY)
:[[Hohenleuben]] (TH)
:[[Hohenmölsen]] (ST)
:[[Hohen Neuendorf]] (BB)
:[[Hohenstein-Ernstthal]] (SN)
:[[Hohnstein (Sächsische Schweiz)|Hohnstein]] (SN)
:[[Höhr-Grenzhausen]] (RP)
:[[Hollfeld]] (BY)
:[[Holzgerlingen]] (BW)
:[[Holzminden]] (NI)
:[[Homberg (Efze)]] (HE)
:[[Homberg (Ohm)]] (HE)
:[[Homburg]] (SL)
:[[Horb am Neckar]] (BW)
:[[Hornbach]] (RP)
:[[Horn-Bad Meinberg]] (NW)
:[[Hornberg]] (BW)
:[[Hörstel]] (NW)
:[[Horstmar]] (NW)
:[[Höxter]] (NW)
:[[Hoya]] (NI)
:[[Hoyerswerda]] (SN)
:[[Hückelhoven]] (NW)
:[[Hückeswagen]] (NW)
:[[Hüfingen]] (BW)
:[[Hünfeld]] (HE)
:[[Hungen]] (HE)
:[[Hürth]] (NW)
:[[Husum]] (SH)
|}

=== I ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Ibbenbüren]] (NW)
:[[Ichenhausen]] (BY)
:[[Idar-Oberstein]] (RP)
:[[Idstein]] (HE)
:[[Illertissen]] (BY)
:[[Ilmenau]] (TH)
| width="400" valign="top" |
:[[Ilsenburg (Harz)]] (ST)
:[[Ilshofen]] (BW)
:[[Immenhausen]] (HE)
:[[Immenstadt im Allgäu]] (BY)
:[[Ingelfingen]] (BW)
:[[Ingelheim am Rhein]] (RP)
| width="400" valign="top" |
:[[Ingolstadt]] (BY)
:[[Iphofen]] (BY)
:[[Iserlohn]] (NW)
:[[Isny im Allgäu]] (BW)
:[[Isselburg]] (NW)
:[[Itzehoe]] (SH)
|}

=== J ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Jarmen]] (MV)
:[[Jena]] (TH)
:[[Jerichow]] (ST)
:[[Jessen (Elster)]] (ST)
| width="400" valign="top" |
:[[Jever]] (NI)
:[[Joachimsthal]] (BB)
:[[Johanngeorgenstadt]] (SN)
:[[Jöhstadt]] (SN)
| width="400" valign="top" |
:[[Jüchen]] (NW)
:[[Jülich]] (NW)
:[[Jüterbog]] (BB)
|}

=== K ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Kaarst]] (NW)
:[[Kahla]] (TH)
:[[Kaisersesch]] (RP)
:[[Kaiserslautern]] (RP)
:[[Kalbe (Milde)]] (ST)
:[[Kalkar]] (NW)
:[[Kaltenkirchen]] (SH)
:[[Kaltennordheim]] (TH)
:[[Kamen]] (NW)
:[[Kamenz]] (SN)
:[[Kamp-Lintfort]] (NW)
:[[Kandel (Pfalz)|Kandel]] (RP)
:[[Kandern]] (BW)
:[[Kappeln]] (SH)
:[[Karben]] (HE)
:[[Karlsruhe]] (BW)
:[[Karlstadt]] (BY)
:[[Kassel]] (HE)
:[[Kastellaun]] (RP)
:[[Katzenelnbogen]] (RP)
:[[Kaub]] (RP)
:[[Kaufbeuren]] (BY)
:[[Kehl]] (BW)
:[[Kelbra (Kyffhäuser)]] (ST)
:[[Kelheim]] (BY)
:[[Kelkheim (Taunus)]] (HE)
:[[Kellinghusen]] (SH)
:[[Kelsterbach]] (HE)
:[[Kemberg]] (ST)
:[[Kemnath]] (BY)
:[[Kempen]] (NW)
:[[Kempten (Allgäu)]] (BY)
:[[Kenzingen]] (BW)
| width="400" valign="top" |
:[[Kerpen]] (NW)
:[[Ketzin/Havel]] (BB)
:[[Kevelaer]] (NW)
:[[Kiel]] (SH, Landeshauptstadt)
:[[Kierspe]] (NW)
:[[Kirchberg (Sachsen)|Kirchberg]] (SN)
:[[Kirchberg an der Jagst]] (BW)
:[[Kirchberg (Hunsrück)]] (RP)
:[[Kirchen (Sieg)]] (RP)
:[[Kirchenlamitz]] (BY)
:[[Kirchhain]] (HE)
:[[Kirchheimbolanden]] (RP)
:[[Kirchheim unter Teck]] (BW)
:[[Kirn]] (RP)
:[[Kirtorf]] (HE)
:[[Kitzingen]] (BY)
:[[Kitzscher]] (SN)
:[[Kleve]] (NW)
:[[Klingenberg am Main]] (BY)
:[[Klingenthal]] (SN)
:[[Klötze]] (ST)
:[[Klütz]] (MV)
:[[Knittlingen]] (BW)
:[[Koblenz]] (RP)
:[[Kolbermoor]] (BY)
:[[Kölleda]] (TH)
:[[Köln]] (NW)
:[[Königsberg in Bayern]] (BY)
:[[Königsbrück]] (SN)
:[[Königsbrunn]] (BY)
:[[Königsee]] (TH)
:[[Königslutter am Elm]] (NI)
:[[Königstein im Taunus]] (HE)
| width="400" valign="top" |
:[[Königstein (Sächsische Schweiz)]] (SN)
:[[Königswinter]] (NW)
:[[Königs Wusterhausen]] (BB)
:[[Könnern]] (ST)
:[[Konstanz]] (BW)
:[[Konz]] (RP)
:[[Korbach]] (HE)
:[[Korntal-Münchingen]] (BW)
:[[Kornwestheim]] (BW)
:[[Korschenbroich]] (NW)
:[[Köthen (Anhalt)]] (ST)
:[[Kraichtal]] (BW)
:[[Krakow am See]] (MV)
:[[Kranichfeld]] (TH)
:[[Krautheim (Jagst)|Krautheim]] (BW)
:[[Krefeld]] (NW)
:[[Kremmen]] (BB)
:[[Krempe (Steinburg)|Krempe]] (SH)
:[[Kreuztal]] (NW)
:[[Kronach]] (BY)
:[[Kronberg im Taunus]] (HE)
:[[Kröpelin]] (MV)
:[[Kroppenstedt]] (ST)
:[[Krumbach (Schwaben)]] (BY)
:[[Kühlungsborn]] (MV)
:[[Kulmbach]] (BY)
:[[Külsheim]] (BW)
:[[Künzelsau]] (BW)
:[[Kupferberg]] (BY)
:[[Kuppenheim]] (BW)
:[[Kusel]] (RP)
:[[Kyllburg]] (RP)
:[[Kyritz]] (BB)
|}

=== L ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Laage]] (MV)
:[[Laatzen]] (NI)
:[[Ladenburg]] (BW)
:[[Lage (Lippe)|Lage]] (NW)
:[[Lahnstein]] (RP)
:[[Lahr/Schwarzwald]] (BW)
:[[Laichingen]] (BW)
:[[Lambrecht (Pfalz)]] (RP)
:[[Lampertheim]] (HE)
:[[Landau an der Isar]] (BY)
:[[Landau in der Pfalz]] (RP)
:[[Landsberg am Lech]] (BY)
:[[Landsberg (Saalekreis)|Landsberg]] (ST)
:[[Landshut]] (BY)
:[[Landstuhl]] (RP)
:[[Langelsheim]] (NI)
:[[Langen (Hessen)|Langen]] (HE)
:[[Langenau]] (BW)
:[[Langenburg]] (BW)
:[[Langenfeld (Rheinland)]] (NW)
:[[Langenhagen]] (NI)
:[[Langenselbold]] (HE)
:[[Langenzenn]] (BY)
:[[Lassan]] (MV)
:[[Laubach]] (HE)
:[[Laucha an der Unstrut]] (ST)
:[[Lauchhammer]] (BB)
:[[Lauchheim]] (BW)
:[[Lauda-Königshofen]] (BW)
:[[Lauenburg/Elbe]] (SH)
:[[Lauf an der Pegnitz]] (BY)
:[[Laufen (Salzach)|Laufen]] (BY)
:[[Laufenburg (Baden)]] (BW)
:[[Lauffen am Neckar]] (BW)
:[[Lauingen (Donau)]] (BY)
:[[Laupheim]] (BW)
:[[Lauscha]] (TH)
:[[Lauta]] (SN)
:[[Lauter-Bernsbach]] (SN)
:[[Lauterbach (Hessen)|Lauterbach]] (HE)
:[[Lauterecken]] (RP)
:[[Lauterstein]] (BW)
:[[Lebach]] (SL)
:[[Lebus]] (BB)
| width="400" valign="top" |
:[[Leer (Ostfriesland)]] (NI)
:[[Lehesten (Thüringer Wald)|Lehesten]] (TH)
:[[Lehrte]] (NI)
:[[Leichlingen (Rheinland)]] (NW)
:[[Leimen (Baden)|Leimen]] (BW)
:[[Leinefelde-Worbis]] (TH)
:[[Leinfelden-Echterdingen]] (BW)
:[[Leingarten]] (BW)
:[[Leipheim]] (BY)
:[[Leipzig]] (SN)
:[[Leisnig]] (SN)
:[[Lemgo]] (NW)
:[[Lengenfeld (Vogtland)|Lengenfeld]] (SN)
:[[Lengerich (Westfalen)|Lengerich]] (NW)
:[[Lennestadt]] (NW)
:[[Lenzen (Elbe)]] (BB)
:[[Leonberg]] (BW)
:[[Leun]] (HE)
:[[Leuna]] (ST)
:[[Leutenberg]] (TH)
:[[Leutershausen]] (BY)
:[[Leutkirch im Allgäu]] (BW)
:[[Leverkusen]] (NW)
:[[Lich]] (HE)
:[[Lichtenau (Baden)|Lichtenau]] (BW)
:[[Lichtenau (Westfalen)|Lichtenau]] (NW)
:[[Lichtenberg (Oberfranken)|Lichtenberg]] (BY)
:[[Lichtenfels (Oberfranken)|Lichtenfels]] (BY)
:[[Lichtenfels (Hessen)|Lichtenfels]] (HE)
:[[Lichtenstein/Sa.]] (SN)
:[[Liebenau (Hessen)|Liebenau]] (HE)
:[[Liebenwalde]] (BB)
:[[Lieberose]] (BB)
:[[Liebstadt]] (SN)
:[[Limbach-Oberfrohna]] (SN)
:[[Limburg an der Lahn]] (HE)
:[[Lindau (Bodensee)]] (BY)
:[[Linden (Hessen)|Linden]] (HE)
:[[Lindenberg im Allgäu]] (BY)
:[[Lindenfels]] (HE)
:[[Lindow (Mark)]] (BB)
:[[Lingen (Ems)]] (NI)
:[[Linnich]] (NW)
| width="400" valign="top" |
:[[Linz am Rhein]] (RP)
:[[Lippstadt]] (NW)
:[[Löbau]] (SN)
:[[Löffingen]] (BW)
:[[Lohmar]] (NW)
:[[Lohne (Oldenburg)]] (NI)
:[[Löhne]] (NW)
:[[Lohr am Main]] (BY)
:[[Loitz]] (MV)
:[[Lollar]] (HE)
:[[Lommatzsch]] (SN)
:[[Löningen]] (NI)
:[[Lorch (Württemberg)|Lorch]] (BW)
:[[Lorch (Rheingau)|Lorch]] (HE)
:[[Lörrach]] (BW)
:[[Lorsch]] (HE)
:[[Lößnitz (Erzgebirge)|Lößnitz]] (SN)
:[[Löwenstein]] (BW)
:[[Lübbecke]] (NW)
:[[Lübben (Spreewald)]] (BB)
:[[Lübbenau/Spreewald]] (BB)
:[[Lübeck]], Hansestadt (SH)
:[[Lübtheen]] (MV)
:[[Lübz]] (MV)
:[[Lüchow (Wendland)]] (NI)
:[[Lucka]] (TH)
:[[Luckau]] (BB)
:[[Luckenwalde]] (BB)
:[[Lüdenscheid]] (NW)
:[[Lüdinghausen]] (NW)
:[[Ludwigsburg]] (BW)
:[[Ludwigsfelde]] (BB)
:[[Ludwigshafen am Rhein]] (RP)
:[[Ludwigslust]] (MV)
:[[Ludwigsstadt]] (BY)
:[[Lugau]] (SN)
:[[Lügde]] (NW)
:[[Lüneburg]], Hansestadt (NI)
:[[Lünen]] (NW)
:[[Lunzenau]] (SN)
:[[Lütjenburg]] (SH)
:[[Lützen]] (ST)
:[[Lychen]] (BB)
|}

=== M ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Magdala]] (TH)
:[[Magdeburg]] (ST, Landeshauptstadt)
:[[Mahlberg]] (BW)
:[[Mainbernheim]] (BY)
:[[Mainburg]] (BY)
:[[Maintal]] (HE)
:[[Mainz]] (RP, Landeshauptstadt)
:[[Malchin]] (MV)
:[[Malchow]] (MV)
:[[Manderscheid]] (RP)
:[[Mannheim]] (BW)
:[[Mansfeld]] (ST)
:[[Marbach am Neckar]] (BW)
:[[Marburg]] (HE)
:[[Marienberg]] (SN)
:[[Marienmünster]] (NW)
:[[Markdorf]] (BW)
:[[Markgröningen]] (BW)
:[[Märkisch Buchholz]] (BB)
:[[Markkleeberg]] (SN)
:[[Markneukirchen]] (SN)
:[[Markranstädt]] (SN)
:[[Marktbreit]] (BY)
:[[Marktheidenfeld]] (BY)
:[[Marktleuthen]] (BY)
:[[Marktoberdorf]] (BY)
:[[Marktredwitz]] (BY)
:[[Marktsteft]] (BY)
:[[Marl]] (NW)
:[[Marlow]] (MV)
:[[Marne (Holstein)|Marne]] (SH)
:[[Marsberg]] (NW)
:[[Maulbronn]] (BW)
:[[Maxhütte-Haidhof]] (BY)
:[[Mayen]] (RP)
:[[Mechernich]] (NW)
:[[Meckenheim (Rheinland)|Meckenheim]] (NW)
| width="400" valign="top" |
:[[Medebach]] (NW)
:[[Meerane]] (SN)
:[[Meerbusch]] (NW)
:[[Meersburg]] (BW)
:[[Meinerzhagen]] (NW)
:[[Meiningen]] (TH)
:[[Meisenheim]] (RP)
:[[Meißen]] (SN)
:[[Meldorf]] (SH)
:[[Melle]] (NI)
:[[Mellrichstadt]] (BY)
:[[Melsungen]] (HE)
:[[Memmingen]] (BY)
:[[Menden (Sauerland)]] (NW)
:[[Mendig]] (RP)
:[[Mengen]] (BW)
:[[Meppen]] (NI)
:[[Merkendorf (Mittelfranken)|Merkendorf]] (BY)
:[[Merseburg]] (ST)
:[[Merzig]] (SL)
:[[Meschede]] (NW)
:[[Meßkirch]] (BW)
:[[Meßstetten]] (BW)
:[[Mettmann]] (NW)
:[[Metzingen]] (BW)
:[[Meuselwitz]] (TH)
:[[Meyenburg]] (BB)
:[[Michelstadt]] (HE)
:[[Miesbach]] (BY)
:[[Miltenberg]] (BY)
:[[Mindelheim]] (BY)
:[[Minden]] (NW)
:[[Mirow]] (MV)
:[[Mittenwalde]] (BB)
:[[Mitterteich]] (BY)
:[[Mittweida]] (SN)
:[[Möckern]] (ST)
| width="400" valign="top" |
:[[Möckmühl]] (BW)
:[[Moers]] (NW)
:[[Mölln]] (SH)
:[[Mönchengladbach]] (NW)
:[[Monheim (Schwaben)|Monheim]] (BY)
:[[Monheim am Rhein]] (NW)
:[[Monschau]] (NW)
:[[Montabaur]] (RP)
:[[Moosburg an der Isar]] (BY)
:[[Mörfelden-Walldorf]] (HE)
:[[Moringen]] (NI)
:[[Mosbach]] (BW)
:[[Mössingen]] (BW)
:[[Mücheln (Geiseltal)]] (ST)
:[[Mügeln]] (SN)
:[[Mühlacker]] (BW)
:[[Mühlberg/Elbe]] (BB)
:[[Mühldorf am Inn]] (BY)
:[[Mühlhausen/Thüringen]] (TH)
:[[Mühlheim am Main]] (HE)
:[[Mühlheim an der Donau]] (BW)
:[[Mülheim an der Ruhr]] (NW)
:[[Mülheim-Kärlich]] (RP)
:[[Müllheim (Baden)|Müllheim]] (BW)
:[[Müllrose]] (BB)
:[[Münchberg]] (BY)
:[[Müncheberg]] (BB)
:[[München]] (BY, Landeshauptstadt)
:[[Münchenbernsdorf]] (TH)
:[[Munderkingen]] (BW)
:[[Münnerstadt]] (BY)
:[[Münsingen (Württemberg)|Münsingen]] (BW)
:[[Munster]] (NI)
:[[Münster]] (NW)
:[[Münstermaifeld]] (RP)
:[[Münzenberg]] (HE)
:[[Murrhardt]] (BW)
|}

=== N ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Nabburg]] (BY)
:[[Nagold]] (BW)
:[[Naila]] (BY)
:[[Nassau (Lahn)|Nassau]] (RP)
:[[Nastätten]] (RP)
:[[Nauen]] (BB)
:[[Naumburg (Hessen)|Naumburg]] (HE)
:[[Naumburg (Saale)]] (ST)
:[[Naunhof]] (SN)
:[[Nebra (Unstrut)]] (ST)
:[[Neckarbischofsheim]] (BW)
:[[Neckargemünd]] (BW)
:[[Neckarsteinach]] (HE)
:[[Neckarsulm]] (BW)
:[[Neresheim]] (BW)
:[[Netphen]] (NW)
:[[Nettetal]] (NW)
:[[Netzschkau]] (SN)
:[[Neu-Anspach]] (HE)
:[[Neubrandenburg]] (MV)
:[[Neubukow]] (MV)
:[[Neubulach]] (BW)
:[[Neuburg an der Donau]] (BY)
:[[Neudenau]] (BW)
:[[Neuenbürg]] (BW)
:[[Neuenburg am Rhein]] (BW)
:[[Neuenhaus]] (NI)
:[[Neuenrade]] (NW)
:[[Neuenstadt am Kocher]] (BW)
:[[Neuenstein (Hohenlohe)|Neuenstein]] (BW)
:[[Neuerburg]] (RP)
:[[Neuffen]] (BW)
:[[Neuhaus am Rennweg]] (TH)
| width="400" valign="top" |
:[[Neu-Isenburg]] (HE)
:[[Neukalen]] (MV)
:[[Neukirchen (Knüll)|Neukirchen]] (HE)
:[[Neukirchen-Vluyn]] (NW)
:[[Neukloster]] (MV)
:[[Neumark (bei Weimar)|Neumark]] (TH)
:[[Neumarkt in der Oberpfalz]] (BY)
:[[Neumarkt-Sankt Veit]] (BY)
:[[Neumünster]] (SH)
:[[Neunburg vorm Wald]] (BY)
:[[Neunkirchen (Saar)|Neunkirchen]] (SL)
:[[Neuötting]] (BY)
:[[Neuruppin]] (BB)
:[[Neusalza-Spremberg]] (SN)
:[[Neusäß]] (BY)
:[[Neuss]] (NW)
:[[Neustadt am Kulm]] (BY)
:[[Neustadt am Rübenberge]] (NI)
:[[Neustadt an der Aisch]] (BY)
:[[Neustadt an der Donau]] (BY)
:[[Neustadt an der Orla]] (TH)
:[[Neustadt an der Waldnaab]] (BY)
:[[Neustadt an der Weinstraße]] (RP)
:[[Neustadt bei Coburg]] (BY)
:[[Neustadt (Dosse)]] (BB)
:[[Neustadt-Glewe]] (MV)
:[[Neustadt (Hessen)]] (HE)
:[[Neustadt in Holstein]] (SH)
:[[Neustadt in Sachsen]] (SN)
:[[Neustrelitz]] (MV)
:[[Neutraubling]] (BY)
:[[Neu-Ulm]] (BY)
| width="400" valign="top" |
:[[Neuwied]] (RP)
:[[Nidda]] (HE)
:[[Niddatal]] (HE)
:[[Nidderau]] (HE)
:[[Nideggen]] (NW)
:[[Niebüll]] (SH)
:[[Niedenstein]] (HE)
:[[Niederkassel]] (NW)
:[[Niedernhall]] (BW)
:[[Nieder-Olm]] (RP)
:[[Niederstetten]] (BW)
:[[Niederstotzingen]] (BW)
:[[Nieheim]] (NW)
:[[Niemegk]] (BB)
:[[Nienburg (Saale)]] (ST)
:[[Nienburg/Weser]] (NI)
:[[Nierstein]] (RP)
:[[Niesky]] (SN)
:[[Nittenau]] (BY)
:[[Norden (Ostfriesland)|Norden]] (NI)
:[[Nordenham]] (NI)
:[[Norderney]] (NI)
:[[Norderstedt]] (SH)
:[[Nordhausen]] (TH)
:[[Nordhorn]] (NI)
:[[Nördlingen]] (BY)
:[[Northeim]] (NI)
:[[Nortorf]] (SH)
:[[Nossen]] (SN)
:[[Nottertal-Heilinger Höhen]] (TH)
:[[Nürnberg]] (BY)
:[[Nürtingen]] (BW)
|}

=== O ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Oberasbach]] (BY)
:[[Oberharz am Brocken]] (ST)
:[[Oberhausen]] (NW)
:[[Oberhof]] (TH)
:[[Oberkirch (Baden)|Oberkirch]] (BW)
:[[Oberkochen]] (BW)
:[[Oberlungwitz]] (SN)
:[[Obermoschel]] (RP)
:[[Obernburg am Main]] (BY)
:[[Oberndorf am Neckar]] (BW)
:[[Obernkirchen]] (NI)
:[[Ober-Ramstadt]] (HE)
:[[Oberriexingen]] (BW)
:[[Obertshausen]] (HE)
:[[Oberursel (Taunus)]] (HE)
:[[Oberviechtach]] (BY)
:[[Oberwesel]] (RP)
:[[Oberwiesenthal]] (SN)
:[[Oberzent]] (HE)
:[[Ochsenfurt]] (BY)
:[[Ochsenhausen]] (BW)
:[[Ochtrup]] (NW)
:[[Oderberg]] (BB)
:[[Oebisfelde-Weferlingen]] (ST)
| width="400" valign="top" |
:[[Oederan]] (SN)
:[[Oelde]] (NW)
:[[Oelsnitz/Erzgeb.]] (SN)
:[[Oelsnitz/Vogtl.]] (SN)
:[[Oer-Erkenschwick]] (NW)
:[[Oerlinghausen]] (NW)
:[[Oestrich-Winkel]] (HE)
:[[Oettingen in Bayern]] (BY)
:[[Offenbach am Main]] (HE)
:[[Offenburg]] (BW)
:[[Ohrdruf]] (TH)
:[[Öhringen]] (BW)
:[[Olbernhau]] (SN)
:[[Olching]] (BY)
:[[Oldenburg (Oldb)]] (NI)
:[[Oldenburg in Holstein]] (SH)
:[[Olfen]] (NW)
:[[Olpe]] (NW)
:[[Olsberg]] (NW)
:[[Oppenau]] (BW)
:[[Oppenheim]] (RP)
:[[Oranienbaum-Wörlitz]] (ST)
:[[Oranienburg]] (BB)
:[[Orlamünde]] (TH)
| width="400" valign="top" |
:[[Ornbau]] (BY)
:[[Ortenberg (Hessen)|Ortenberg]] (HE)
:[[Ortrand]] (BB)
:[[Oschatz]] (SN)
:[[Oschersleben (Bode)]] (ST)
:[[Osnabrück]] (NI)
:[[Osterburg (Altmark)]] (ST)
:[[Osterburken]] (BW)
:[[Osterfeld (Sachsen-Anhalt)|Osterfeld]] (ST)
:[[Osterhofen]] (BY)
:[[Osterholz-Scharmbeck]] (NI)
:[[Osterode am Harz]] (NI)
:[[Osterwieck]] (ST)
:[[Ostfildern]] (BW)
:[[Ostheim vor der Rhön]] (BY)
:[[Osthofen]] (RP)
:[[Östringen]] (BW)
:[[Ostritz]] (SN)
:[[Otterberg]] (RP)
:[[Otterndorf]] (NI)
:[[Ottweiler]] (SL)
:[[Overath]] (NW)
:[[Owen]] (BW)
|}

=== P ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Paderborn]] (NW)
:[[Papenburg]] (NI)
:[[Pappenheim]] (BY)
:[[Parchim]] (MV)
:[[Parsberg]] (BY)
:[[Pasewalk]] (MV)
:[[Passau]] (BY)
:[[Pattensen]] (NI)
:[[Pausa-Mühltroff]] (SN)
:[[Pegau]] (SN)
:[[Pegnitz (Stadt)|Pegnitz]] (BY)
:[[Peine]] (NI)
:[[Peitz]] (BB)
:[[Penig]] (SN)
:[[Penkun]] (MV)
:[[Penzberg]] (BY)
:[[Penzlin]] (MV)
:[[Perleberg]] (BB)
:[[Petershagen]] (NW)
:[[Pfaffenhofen an der Ilm]] (BY)
| width="400" valign="top" |
:[[Pfarrkirchen]] (BY)
:[[Pforzheim]] (BW)
:[[Pfreimd]] (BY)
:[[Pfullendorf]] (BW)
:[[Pfullingen]] (BW)
:[[Pfungstadt]] (HE)
:[[Philippsburg]] (BW)
:[[Pinneberg]] (SH)
:[[Pirmasens]] (RP)
:[[Pirna]] (SN)
:[[Plattling]] (BY)
:[[Plau am See]] (MV)
:[[Plaue]] (TH)
:[[Plauen]] (SN)
:[[Plettenberg]] (NW)
:[[Pleystein]] (BY)
:[[Plochingen]] (BW)
:[[Plön]] (SH)
:[[Pockau-Lengefeld]] (SN)
:[[Pocking]] (BY)
| width="400" valign="top" |
:[[Pohlheim]] (HE)
:[[Polch]] (RP)
:[[Porta Westfalica (Stadt)|Porta Westfalica]] (NW)
:[[Pößneck]] (TH)
:[[Potsdam]] (BB, Landeshauptstadt)
:[[Pottenstein (Oberfranken)|Pottenstein]] (BY)
:[[Preetz]] (SH)
:[[Premnitz]] (BB)
:[[Prenzlau]] (BB)
:[[Pressath]] (BY)
:[[Preußisch Oldendorf]] (NW)
:[[Prichsenstadt]] (BY)
:[[Pritzwalk]] (BB)
:[[Prüm]] (RP)
:[[Puchheim]] (BY)
:[[Pulheim]] (NW)
:[[Pulsnitz]] (SN)
:[[Putbus]] (MV)
:[[Putlitz]] (BB)
:[[Püttlingen]] (SL)
|}

=== Q ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Quakenbrück]] (NI)
:[[Quedlinburg]] (ST)
| width="400" valign="top" |
:[[Querfurt]] (ST)
| width="400" valign="top" |
:[[Quickborn]] (SH)
|}

=== R ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Rabenau (Sachsen)|Rabenau]] (SN)
:[[Radeberg]] (SN)
:[[Radebeul]] (SN)
:[[Radeburg]] (SN)
:[[Radevormwald]] (NW)
:[[Radolfzell am Bodensee]] (BW)
:[[Raguhn-Jeßnitz]] (ST)
:[[Rahden]] (NW)
:[[Rain (Lech)|Rain]] (BY)
:[[Ramstein-Miesenbach]] (RP)
:[[Ranis]] (TH)
:[[Ransbach-Baumbach]] (RP)
:[[Rastatt]] (BW)
:[[Rastenberg]] (TH)
:[[Rathenow]] (BB)
:[[Ratingen]] (NW)
:[[Ratzeburg]] (SH)
:[[Rauenberg]] (BW)
:[[Raunheim]] (HE)
:[[Rauschenberg]] (HE)
:[[Ravensburg]] (BW)
:[[Ravenstein]] (BW)
:[[Recklinghausen]] (NW)
:[[Rees]] (NW)
:[[Regen (Stadt)|Regen]] (BY)
:[[Regensburg]] (BY)
:[[Regis-Breitingen]] (SN)
:[[Rehau]] (BY)
:[[Rehburg-Loccum]] (NI)
:[[Rehna]] (MV)
:[[Reichelsheim (Wetterau)]] (HE)
:[[Reichenbach im Vogtland]] (SN)
:[[Reichenbach/O.L.]] (SN)
:[[Reinbek]] (SH)
:[[Reinfeld (Holstein)]] (SH)
:[[Reinheim]] (HE)
:[[Remagen]] (RP)
| width="400" valign="top" |
:[[Remscheid]] (NW)
:[[Remseck am Neckar]] (BW)
:[[Renchen]] (BW)
:[[Rendsburg]] (SH)
:[[Rennerod]] (RP)
:[[Renningen]] (BW)
:[[Rerik]] (MV)
:[[Rethem (Aller)]] (NI)
:[[Reutlingen]] (BW)
:[[Rheda-Wiedenbrück]] (NW)
:[[Rhede]] (NW)
:[[Rheinau (Baden)|Rheinau]] (BW)
:[[Rheinbach]] (NW)
:[[Rheinberg]] (NW)
:[[Rheinböllen]] (RP)
:[[Rheine]] (NW)
:[[Rheinfelden (Baden)]] (BW)
:[[Rheinsberg]] (BB)
:[[Rheinstetten]] (BW)
:[[Rhens]] (RP)
:[[Rhinow]] (BB)
:[[Ribnitz-Damgarten]] (MV)
:[[Richtenberg]] (MV)
:[[Riedenburg]] (BY)
:[[Riedlingen]] (BW)
:[[Riedstadt]] (HE)
:[[Rieneck]] (BY)
:[[Riesa]] (SN)
:[[Rietberg]] (NW)
:[[Rinteln]] (NI)
:[[Röbel/Müritz]] (MV)
:[[Rochlitz]] (SN)
:[[Rockenhausen]] (RP)
:[[Rodalben]] (RP)
:[[Rodenberg]] (NI)
:[[Rödental]] (BY)
:[[Rödermark]] (HE)
| width="400" valign="top" |
:[[Rodewisch]] (SN)
:[[Rodgau]] (HE)
:[[Roding]] (BY)
:[[Römhild]] (TH)
:[[Romrod]] (HE)
:[[Ronneburg (Thüringen)|Ronneburg]] (TH)
:[[Ronnenberg]] (NI)
:[[Rosbach vor der Höhe]] (HE)
:[[Rosenfeld]] (BW)
:[[Rosenheim]] (BY)
:[[Rosenthal (Hessen)|Rosenthal]] (HE)
:[[Rösrath]] (NW)
:[[Roßleben-Wiehe]] (TH)
:[[Roßwein]] (SN)
:[[Rostock]], Hansestadt (MV)
:[[Rotenburg an der Fulda]] (HE)
:[[Rotenburg (Wümme)]] (NI)
:[[Roth]] (BY)
:[[Rötha]] (SN)
:[[Röthenbach an der Pegnitz]] (BY)
:[[Rothenburg/Oberlausitz|Rothenburg/O.L.]] (SN)
:[[Rothenburg ob der Tauber]] (BY)
:[[Rothenfels]] (BY)
:[[Rottenburg am Neckar]] (BW)
:[[Rottenburg an der Laaber|Rottenburg a.d.Laaber]] (BY)
:[[Röttingen]] (BY)
:[[Rottweil]] (BW)
:[[Rötz]] (BY)
:[[Rüdesheim am Rhein]] (HE)
:[[Rudolstadt]] (TH)
:[[Ruhla]] (TH)
:[[Ruhland]] (BB)
:[[Runkel]] (HE)
:[[Rüsselsheim am Main]] (HE)
:[[Rutesheim]] (BW)
:[[Rüthen]] (NW)
|}

=== S ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Saalburg-Ebersdorf]] (TH)
:[[Saalfeld/Saale]] (TH)
:[[Saarbrücken]] (SL, Landeshauptstadt)
:[[Saarburg]] (RP)
:[[Saarlouis]] (SL)
:[[Sachsenhagen]] (NI)
:[[Sachsenheim]] (BW)
:[[Salzgitter]] (NI)
:[[Salzkotten]] (NW)
:[[Salzwedel]], Hansestadt (ST)
:[[Sandau (Elbe)]] (ST)
:[[Sandersdorf-Brehna]] (ST)
:[[Sangerhausen]] (ST)
:[[Sankt Augustin]] (NW)
:[[Sankt Goar]] (RP)
:[[Sankt Goarshausen]] (RP)
:[[Sarstedt]] (NI)
:[[Sassenberg]] (NW)
:[[Sassnitz]] (MV)
:[[Sayda]] (SN)
:[[Schalkau]] (TH)
:[[Schauenstein]] (BY)
:[[Scheer]] (BW)
:[[Scheibenberg]] (SN)
:[[Scheinfeld]] (BY)
:[[Schelklingen]] (BW)
:[[Schenefeld (Kreis Pinneberg)|Schenefeld]] (SH)
:[[Scheßlitz]] (BY)
:[[Schieder-Schwalenberg]] (NW)
:[[Schifferstadt]] (RP)
:[[Schillingsfürst]] (BY)
:[[Schiltach]] (BW)
:[[Schirgiswalde-Kirschau]] (SN)
:[[Schkeuditz]] (SN)
:[[Schkölen]] (TH)
:[[Schleiden]] (NW)
:[[Schleiz]] (TH)
:[[Schleswig]] (SH)
:[[Schlettau]] (SN)
:[[Schleusingen]] (TH)
:[[Schlieben]] (BB)
:[[Schlitz (Vogelsbergkreis)|Schlitz]] (HE)
:[[Schloß Holte-Stukenbrock]] (NW)
:[[Schlüchtern]] (HE)
:[[Schlüsselfeld]] (BY)
:[[Schmalkalden]] (TH)
:[[Schmallenberg]] (NW)
:[[Schmölln]] (TH)
:[[Schnackenburg]] (NI)
:[[Schnaittenbach]] (BY)
:[[Schneeberg (Erzgebirge)|Schneeberg]] (SN)
:[[Schneverdingen]] (NI)
:[[Schömberg (Zollernalbkreis)|Schömberg]] (BW)
:[[Schönau (Odenwald)|Schönau]] (BW)
:[[Schönau im Schwarzwald]] (BW)
:[[Schönberg (Mecklenburg)|Schönberg]] (MV)
:[[Schönebeck (Elbe)]] (ST)
:[[Schöneck/Vogtl.]] (SN)
:[[Schönewalde]] (BB)
:[[Schongau]] (BY)
:[[Schöningen]] (NI)
:[[Schönsee]] (BY)
:[[Schönwald (Bayern)|Schönwald]] (BY)
:[[Schopfheim]] (BW)
:[[Schöppenstedt]] (NI)
:[[Schorndorf]] (BW)
:[[Schortens]] (NI)
| width="400" valign="top" |
:[[Schotten (Stadt)|Schotten]] (HE)
:[[Schramberg]] (BW)
:[[Schraplau]] (ST)
:[[Schriesheim]] (BW)
:[[Schrobenhausen]] (BY)
:[[Schrozberg]] (BW)
:[[Schüttorf]] (NI)
:[[Schwaan]] (MV)
:[[Schwabach]] (BY)
:[[Schwäbisch Gmünd]] (BW)
:[[Schwäbisch Hall]] (BW)
:[[Schwabmünchen]] (BY)
:[[Schwaigern]] (BW)
:[[Schwalbach am Taunus]] (HE)
:[[Schwalmstadt]] (HE)
:[[Schwandorf]] (BY)
:[[Schwanebeck]] (ST)
:[[Schwarzatal]] (TH)
:[[Schwarzenbach am Wald]] (BY)
:[[Schwarzenbach an der Saale]] (BY)
:[[Schwarzenbek]] (SH)
:[[Schwarzenberg/Erzgeb.]] (SN)
:[[Schwarzenborn (Knüll)|Schwarzenborn]] (HE)
:[[Schwarzheide]] (BB)
:[[Schwedt/Oder]] (BB)
:[[Schweich]] (RP)
:[[Schweinfurt]] (BY)
:[[Schwelm]] (NW)
:[[Schwentinental]] (SH)
:[[Schwerin]] (MV, Landeshauptstadt)
:[[Schwerte]] (NW)
:[[Schwetzingen]] (BW)
:[[Sebnitz]] (SN)
:[[Seehausen (Altmark)]] (ST)
:[[Seeland (Sachsen-Anhalt)|Seeland]] (ST)
:[[Seelow]] (BB)
:[[Seelze]] (NI)
:[[Seesen]] (NI)
:[[Sehnde]] (NI)
:[[Seifhennersdorf]] (SN)
:[[Selb]] (BY)
:[[Selbitz (Oberfranken)|Selbitz]] (BY)
:[[Seligenstadt]] (HE)
:[[Selm]] (NW)
:[[Selters (Westerwald)]] (RP)
:[[Senden (Bayern)|Senden]] (BY)
:[[Sendenhorst]] (NW)
:[[Senftenberg]] (BB)
:[[Seßlach]] (BY)
:[[Siegburg]] (NW)
:[[Siegen]] (NW)
:[[Sigmaringen]] (BW)
:[[Simbach am Inn]] (BY)
:[[Simmern/Hunsrück]] (RP)
:[[Sindelfingen]] (BW)
:[[Singen (Hohentwiel)]] (BW)
:[[Sinsheim]] (BW)
:[[Sinzig]] (RP)
:[[Soest]] (NW)
:[[Solingen]] (NW)
:[[Solms]] (HE)
:[[Soltau]] (NI)
:[[Sömmerda]] (TH)
:[[Sondershausen]] (TH)
:[[Sonneberg]] (TH)
:[[Sonnewalde]] (BB)
:[[Sonthofen]] (BY)
| width="400" valign="top" |
:[[Sontra]] (HE)
:[[Spaichingen]] (BW)
:[[Spalt]] (BY)
:[[Spangenberg]] (HE)
:[[Speicher (Eifel)|Speicher]] (RP)
:[[Spenge]] (NW)
:[[Speyer]] (RP)
:[[Spremberg]] (BB)
:[[Springe]] (NI)
:[[Sprockhövel]] (NW)
:[[Stade]] (NI)
:[[Stadtallendorf]] (HE)
:[[Stadtbergen]] (BY)
:[[Stadthagen]] (NI)
:[[Stadtilm]] (TH)
:[[Stadtlohn]] (NW)
:[[Stadtoldendorf]] (NI)
:[[Stadtprozelten]] (BY)
:[[Stadtroda]] (TH)
:[[Stadtsteinach]] (BY)
:[[Stadt Wehlen]] (SN)
:[[Starnberg]] (BY)
:[[Staßfurt]] (ST)
:[[Staufen im Breisgau]] (BW)
:[[Staufenberg (Hessen)|Staufenberg]] (HE)
:[[Stavenhagen]] (MV)
:[[St. Blasien]] (BW)
:[[Stein (Mittelfranken)|Stein]] (BY)
:[[Steinach (Thüringen)|Steinach]] (TH)
:[[Steinau an der Straße]] (HE)
:[[Steinbach-Hallenberg]] (TH)
:[[Steinbach (Taunus)]] (HE)
:[[Steinfurt]] (NW)
:[[Steinheim (Westfalen)|Steinheim]] (NW)
:[[Steinheim an der Murr]] (BW)
:[[Stendal]] (ST)
:[[Sternberg]] (MV)
:[[St. Ingbert]] (SL)
:[[St. Georgen im Schwarzwald]] (BW)
:[[Stockach]] (BW)
:[[Stolberg (Rheinland)]] (NW)
:[[Stollberg/Erzgebirge|Stollberg/Erzgeb.]] (SN)
:[[Stolpen]] (SN)
:[[Storkow (Mark)]] (BB)
:[[Stößen]] (ST)
:[[Straelen]] (NW)
:[[Stralsund]], Hansestadt (MV)
:[[Strasburg (Uckermark)]] (MV)
:[[Straubing]] (BY)
:[[Strausberg]] (BB)
:[[Strehla]] (SN)
:[[Stromberg (Hunsrück)|Stromberg]] (RP)
:[[Stühlingen]] (BW)
:[[Stutensee]] (BW)
:[[Stuttgart]] (BW, Landeshauptstadt)
:[[St. Wendel]] (SL)
:[[Südliches Anhalt]] (ST)
:[[Suhl]] (TH)
:[[Sulingen]] (NI)
:[[Sulz am Neckar]] (BW)
:[[Sulzbach/Saar]] (SL)
:[[Sulzbach-Rosenberg]] (BY)
:[[Sulzburg]] (BW)
:[[Sundern (Sauerland)]] (NW)
:[[Süßen]] (BW)
:[[Syke]] (NI)
|}

=== T ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Tambach-Dietharz]] (TH)
:[[Tangerhütte]] (ST)
:[[Tangermünde]] (ST)
:[[Tann (Rhön)]] (HE)
:[[Tanna]] (TH)
:[[Tauberbischofsheim]] (BW)
:[[Taucha]] (SN)
:[[Taunusstein]] (HE)
:[[Tecklenburg]] (NW)
:[[Tegernsee (Stadt)|Tegernsee]] (BY)
:[[Telgte]] (NW)
:[[Teltow]] (BB)
:[[Templin]] (BB)
:[[Tengen]] (BW)
:[[Tessin (bei Rostock)|Tessin]] (MV)
:[[Teterow]] (MV)
:[[Tettnang]] (BW)
:[[Teublitz]] (BY)
:[[Teuchern]] (ST)
:[[Teupitz]] (BB)
| width="400" valign="top" |
:[[Teuschnitz]] (BY)
:[[Thale]] (ST)
:[[Thalheim/Erzgeb.]] (SN)
:[[Thannhausen (Schwaben)|Thannhausen]] (BY)
:[[Tharandt]] (SN)
:[[Themar]] (TH)
:[[Thum]] (SN)
:[[Tirschenreuth]] (BY)
:[[Titisee-Neustadt]] (BW)
:[[Tittmoning]] (BY)
:[[Todtnau]] (BW)
:[[Töging am Inn]] (BY)
:[[Tönisvorst]] (NW)
:[[Tönning]] (SH)
:[[Torgau]] (SN)
:[[Torgelow]] (MV)
:[[Tornesch]] (SH)
:[[Traben-Trarbach]] (RP)
:[[Traunreut]] (BY)
| width="400" valign="top" |
:[[Traunstein]] (BY)
:[[Trebbin]] (BB)
:[[Trebsen/Mulde]] (SN)
:[[Treffurt]] (TH)
:[[Trendelburg]] (HE)
:[[Treuchtlingen]] (BY)
:[[Treuen]] (SN)
:[[Treuenbrietzen]] (BB)
:[[Triberg im Schwarzwald]] (BW)
:[[Tribsees]] (MV)
:[[Trier]] (RP)
:[[Triptis]] (TH)
:[[Trochtelfingen]] (BW)
:[[Troisdorf]] (NW)
:[[Trossingen]] (BW)
:[[Trostberg]] (BY)
:[[Tübingen]] (BW)
:[[Tuttlingen]] (BW)
:[[Twistringen]] (NI)
|}

=== U ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Übach-Palenberg]] (NW)
:[[Überlingen]] (BW)
:[[Uebigau-Wahrenbrück]] (BB)
:[[Ueckermünde]] (MV)
:[[Uelzen]] (NI)
:[[Uetersen]] (SH)
| width="400" valign="top" |
:[[Uffenheim]] (BY)
:[[Uhingen]] (BW)
:[[Ulm]] (BW)
:[[Ulmen (Eifel)|Ulmen]] (RP)
:[[Ulrichstein]] (HE)
:[[Ummerstadt]] (TH)
| width="400" valign="top" |
:[[Unkel]] (RP)
:[[Unna]] (NW)
:[[Unterschleißheim]] (BY)
:[[Usedom (Stadt)|Usedom]] (MV)
:[[Usingen]] (HE)
:[[Uslar]] (NI)
|}

=== V ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Vacha]] (TH)
:[[Vaihingen an der Enz]] (BW)
:[[Vallendar]] (RP)
:[[Varel]] (NI)
:[[Vechta]] (NI)
:[[Velbert]] (NW)
:[[Velburg]] (BY)
:[[Velden (Pegnitz)|Velden]] (BY)
:[[Vellberg]] (BW)
:[[Velen]] (NW)
:[[Vellmar]] (HE)
:[[Velten]] (BB)
| width="400" valign="top" |
:[[Verden (Aller)]] (NI)
:[[Veringenstadt]] (BW)
:[[Verl]] (NW)
:[[Versmold]] (NW)
:[[Vetschau/Spreewald]] (BB)
:[[Viechtach]] (BY)
:[[Viernheim]] (HE)
:[[Viersen]] (NW)
:[[Villingen-Schwenningen]] (BW)
:[[Vilsbiburg]] (BY)
:[[Vilseck]] (BY)
:[[Vilshofen an der Donau]] (BY)
| width="400" valign="top" |
:[[Visselhövede]] (NI)
:[[Vlotho]] (NW)
:[[Voerde (Niederrhein)]] (NW)
:[[Vogtsburg im Kaiserstuhl]] (BW)
:[[Vohburg an der Donau]] (BY)
:[[Vohenstrauß]] (BY)
:[[Vöhrenbach]] (BW)
:[[Vöhringen (Iller)|Vöhringen]] (BY)
:[[Volkach]] (BY)
:[[Völklingen]] (SL)
:[[Volkmarsen]] (HE)
:[[Vreden]] (NW)
|}

=== W ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="35%" valign="top" |
:[[Wachenheim an der Weinstraße]] (RP)
:[[Wächtersbach]] (HE)
:[[Wadern]] (SL)
:[[Waghäusel]] (BW)
:[[Wahlstedt]] (SH)
:[[Waiblingen]] (BW)
:[[Waibstadt]] (BW)
:[[Waischenfeld]] (BY)
:[[Waldbröl]] (NW)
:[[Waldeck (Stadt)|Waldeck]] (HE)
:[[Waldenbuch]] (BW)
:[[Waldenburg (Sachsen)|Waldenburg]] (SN)
:[[Waldenburg (Württemberg)|Waldenburg]] (BW)
:[[Waldershof]] (BY)
:[[Waldheim]] (SN)
:[[Waldkappel]] (HE)
:[[Waldkirch]] (BW)
:[[Waldkirchen]] (BY)
:[[Waldkraiburg]] (BY)
:[[Waldmohr]] (RP)
:[[Waldmünchen]] (BY)
:[[Waldsassen]] (BY)
:[[Waldshut-Tiengen]] (BW)
:[[Walldorf (Baden)|Walldorf]] (BW)
:[[Walldürn]] (BW)
:[[Wallenfels]] (BY)
:[[Walsrode]] (NI)
:[[Waltershausen]] (TH)
:[[Waltrop]] (NW)
:[[Wanfried]] (HE)
:[[Wangen im Allgäu]] (BW)
:[[Wanzleben-Börde]] (ST)
:[[Warburg]] (NW)
:[[Waren (Müritz)]] (MV)
:[[Warendorf]] (NW)
:[[Warin]] (MV)
:[[Warstein]] (NW)
:[[Wassenberg]] (NW)
:[[Wasserburg am Inn]] (BY)
:[[Wassertrüdingen]] (BY)
:[[Wasungen]] (TH)
:[[Wedel]] (SH)
:[[Weener]] (NI)
:[[Wegberg]] (NW)
:[[Wegeleben]] (ST)
:[[Wehr (Baden)|Wehr]] (BW)
:[[Weida]] (TH)
:[[Weiden in der Oberpfalz]] (BY)
:[[Weikersheim]] (BW)
:[[Weil am Rhein]] (BW)
:[[Weilburg]] (HE)
:[[Weil der Stadt]] (BW)
:[[Weilheim an der Teck]] (BW)
:[[Weilheim in Oberbayern]] (BY)
| width="400" valign="top" |
:[[Weimar]] (TH)
:[[Weingarten (Württemberg)|Weingarten]] (BW)
:[[Weinheim]] (BW)
:[[Weinsberg]] (BW)
:[[Weinstadt]] (BW)
:[[Weismain]] (BY)
:[[Weißenberg]] (SN)
:[[Weißenburg in Bayern]] (BY)
:[[Weißenfels]] (ST)
:[[Weißenhorn]] (BY)
:[[Weißensee (Thüringen)|Weißensee]] (TH)
:[[Weißenstadt]] (BY)
:[[Weißenthurm]] (RP)
:[[Weißwasser/Oberlausitz|Weißwasser/O.L.]] (SN)
:[[Weiterstadt]] (HE)
:[[Welzheim]] (BW)
:[[Welzow]] (BB)
:[[Wemding]] (BY)
:[[Wendlingen am Neckar]] (BW)
:[[Werben (Elbe)]] (ST)
:[[Werdau]] (SN)
:[[Werder (Havel)]] (BB)
:[[Werdohl]] (NW)
:[[Werl]] (NW)
:[[Werlte]] (NI)
:[[Wermelskirchen]] (NW)
:[[Wernau (Neckar)]] (BW)
:[[Werne]] (NW)
:[[Werneuchen]] (BB)
:[[Wernigerode]] (ST)
:[[Werra-Suhl-Tal]] (TH)
:[[Wertheim]] (BW)
:[[Werther (Westf.)]] (NW)
:[[Wertingen]] (BY)
:[[Wesel]] (NW)
:[[Wesenberg (Mecklenburg)|Wesenberg]] (MV)
:[[Wesselburen]] (SH)
:[[Wesseling]] (NW)
:[[Westerburg]] (RP)
:[[Westerstede]] (NI)
:[[Wetter (Hessen)]] (HE)
:[[Wetter (Ruhr)]] (NW)
:[[Wettin-Löbejün]] (ST)
:[[Wetzlar]] (HE)
:[[Widdern]] (BW)
:[[Wiehl]] (NW)
:[[Wiesbaden]] (HE, Landeshauptstadt)
:[[Wiesmoor]] (NI)
:[[Wiesensteig]] (BW)
:[[Wiesloch]] (BW)
:[[Wildau]] (BB)
:[[Wildberg (Schwarzwald)|Wildberg]] (BW)
:[[Wildenfels]] (SN)
:[[Wildeshausen]] (NI)
| width="400" valign="top" |
:[[Wilhelmshaven]] (NI)
:[[Wilkau-Haßlau]] (SN)
:[[Willebadessen]] (NW)
:[[Willich]] (NW)
:[[Wilsdruff]] (SN)
:[[Wilster]] (SH)
:[[Wilthen]] (SN)
:[[Windischeschenbach]] (BY)
:[[Windsbach]] (BY)
:[[Winnenden]] (BW)
:[[Winsen (Luhe)]] (NI)
:[[Winterberg]] (NW)
:[[Wipperfürth]] (NW)
:[[Wirges]] (RP)
:[[Wismar]], Hansestadt (MV)
:[[Wissen (Stadt)|Wissen]] (RP)
:[[Witten]] (NW)
:[[Lutherstadt Wittenberg|Wittenberg]] (ST)
:[[Wittenberge]] (BB)
:[[Wittenburg]] (MV)
:[[Wittichenau]] (SN)
:[[Wittlich]] (RP)
:[[Wittingen]] (NI)
:[[Wittmund]] (NI)
:[[Wittstock/Dosse]] (BB)
:[[Witzenhausen]] (HE)
:[[Woldegk]] (MV)
:[[Wolfach]] (BW)
:[[Wolfenbüttel]] (NI)
:[[Wolfhagen]] (HE)
:[[Wolframs-Eschenbach]] (BY)
:[[Wolfratshausen]] (BY)
:[[Wolfsburg]] (NI)
:[[Wolfstein]] (RP)
:[[Wolgast]] (MV)
:[[Wolkenstein (Erzgebirge)|Wolkenstein]] (SN)
:[[Wolmirstedt]] (ST)
:[[Worms]] (RP)
:[[Wörrstadt]] (RP)
:[[Wörth am Main]] (BY)
:[[Wörth am Rhein]] (RP)
:[[Wörth an der Donau]] (BY)
:[[Wriezen]] (BB)
:[[Wülfrath]] (NW)
:[[Wunsiedel]] (BY)
:[[Wunstorf]] (NI)
:[[Wuppertal]] (NW)
:[[Würselen]] (NW)
:[[Wurzbach]] (TH)
:[[Würzburg]] (BY)
:[[Wurzen]] (SN)
<!-- Wusterhausen/Dosse ist keine verwaltungsrechtlich selbständige Gemeinde mit Stadtrecht, siehe Artikel Wusterhausen/Dosse (Abschnitt Stadtrecht) und auch Liste ehemaliger Städte in Deutschland (Link am Ende dieses Artikels). -->
:[[Wustrow (Wendland)]] (NI)
:[[Wyk auf Föhr]] (SH)
|}

=== X ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Xanten]] (NW)
| width="400" valign="top" |
| width="400" valign="top" |
|}

=== Z ===
{{TOC}}
{|
| colspan="3" |
|-----
| width="400" valign="top" |
:[[Zahna-Elster]] (ST)
:[[Zarrentin am Schaalsee]] (MV)
:[[Zehdenick]] (BB)
:[[Zeil am Main]] (BY)
:[[Zeitz]] (ST)
:[[Zell am Harmersbach]] (BW)
:[[Zell im Wiesental]] (BW)
:[[Zell (Mosel)]] (RP)
:[[Zella-Mehlis]] (TH)
| width="400" valign="top" |
:[[Zerbst/Anhalt]] (ST)
:[[Zeulenroda-Triebes]] (TH)
:[[Zeven]] (NI)
:[[Ziegenrück]] (TH)
:[[Zierenberg]] (HE)
:[[Ziesar]] (BB)
:[[Zirndorf]] (BY)
:[[Zittau]] (SN)
:[[Zörbig]] (ST)
| width="400" valign="top" |
:[[Zossen]] (BB)
:[[Zschopau]] (SN)
:[[Zülpich]] (NW)
:[[Zweibrücken]] (RP)
:[[Zwenkau]] (SN)
:[[Zwickau]] (SN)
:[[Zwiesel]] (BY)
:[[Zwingenberg (Bergstraße)|Zwingenberg]] (HE)
:[[Zwönitz]] (SN)
|}"""

In [85]:
reg = re.compile("\\:\\[\\[(.*?)\\]\\]")

def find_cities(term: str):
    cities = reg.findall(term)
    if len(cities) == 1:
        return cities[0].split("|")[-1]
    else:
        return None


city_list = wikipedia_list_of_countries.split("\n")
city_list = [find_cities(c) for c in city_list]
city_list = [c.replace("/", " ").lower() for c in city_list if c]

# Check confusion on all german cities

In [86]:
geo_service = GeoLocationFromCoordinatesService(
    ELASTICSEARCH_URL,
    query_builder=CoordinatesLocationAndCountryQuery(fuzzy=True)
)

high_variance_cities = dict()
high_variance_first_result = dict()

for city in tqdm.tqdm(city_list):
    payload = RequestPayload(query=Query(location=city))
    query = geo_service.query_builder.query(payload).query
    results = geo_service.execute(query)["hits"]["hits"]

    geo_points = [extract_geo_point(d) for d in results]
    mean, std = np.mean(geo_points, axis=0), np.std(geo_points, axis=0)

    if np.all(std < 0.1):
        continue  # All results are very close to each other, nothing to check

    if np.any(std > 5):
        high_variance_cities[city] = results

    for r in results:
        gp = np.array(extract_geo_point(r))
        is_highly_variant = np.any(np.abs(gp-mean) > std)
        if is_highly_variant:
            high_variance_first_result[city] = results
        break


100%|██████████| 2054/2054 [12:24<00:00,  2.76it/s]


In [87]:
len(high_variance_cities), len(high_variance_first_result)

(296, 209)

In [88]:
random_city = random.sample(list(high_variance_first_result), 1)[0]
print(random_city)
high_variance_first_result[random_city]

haren (ems)


[{'_index': 'coordinates_v2',
  '_type': 'coordinates',
  '_id': '5655585c0cf2cd954986f7fe',
  '_score': 43.633068,
  '_source': {'location': 'New Haten Haven',
   'geoPoint': {'lon': -72.9250518, 'lat': 41.3082138}}},
 {'_index': 'coordinates_v2',
  '_type': 'coordinates',
  '_id': '55921ebc0cf2b351cf29628c',
  '_score': 34.107346,
  '_source': {'location': 'Haren',
   'geoPoint': {'lon': 7.24142, 'lat': 52.79262}}},
 {'_index': 'coordinates_v2',
  '_type': 'coordinates',
  '_id': '55797e69e4b041dd9a6eceff',
  '_score': 34.107346,
  '_source': {'location': 'Biologisch Centrum Kerklaan 30-al001 Haren',
   'geoPoint': {'lon': 5.58446, 'lat': 51.80307}}},
 {'_index': 'coordinates_v2',
  '_type': 'coordinates',
  '_id': '5583f61fe4b0c8b0aecaf12c',
  '_score': 34.093628,
  '_source': {'location': 'Haren',
   'geoPoint': {'lon': 7.24142, 'lat': 52.79262}}},
 {'_index': 'coordinates_v2',
  '_type': 'coordinates',
  '_id': '56540067e4b00daddb8b6293',
  '_score': 34.093628,
  '_source': {'loca

# No Fuzzy

In [89]:
geo_service = GeoLocationFromCoordinatesService(
    ELASTICSEARCH_URL,
    query_builder=CoordinatesLocationAndCountryQuery(fuzzy=False)
)

nf_high_variance_cities = dict()
nf_high_variance_first_result = dict()

for city in tqdm.tqdm(city_list):
    payload = RequestPayload(query=Query(location=city))
    query = geo_service.query_builder.query(payload).query
    try:
        results = geo_service.execute(query)["hits"]["hits"]

        geo_points = [extract_geo_point(d) for d in results]
        mean, std = np.mean(geo_points, axis=0), np.std(geo_points, axis=0)

        if np.all(std < 0.1):
            continue  # All results are very close to each other, nothing to check

        if np.any(std > 5):
            nf_high_variance_cities[city] = results

        for r in results:
            gp = np.array(extract_geo_point(r))
            is_highly_variant = np.any(np.abs(gp-mean) > std)
            if is_highly_variant:
                nf_high_variance_first_result[city] = results
            break
    except RequestError:
        print(f"Broken request for {city}")
        pass

100%|██████████| 2054/2054 [03:29<00:00,  9.80it/s]


In [90]:
len(nf_high_variance_cities), len(nf_high_variance_first_result)

(170, 58)

In [92]:
random_city = random.sample(list(nf_high_variance_first_result), 1)[0]
print(random_city)
nf_high_variance_first_result[random_city]

weißenberg


[{'_index': 'coordinates_v2',
  '_type': 'coordinates',
  '_id': '558bfeabe4b03b5f46a06fb2',
  '_score': 57.998352,
  '_source': {'location': 'Weißenberg 2',
   'geoPoint': {'lon': 6.676152119893569, 'lat': 51.221532350000004}}},
 {'_index': 'coordinates_v2',
  '_type': 'coordinates',
  '_id': '5b1c3a5346e0fb00127afd03',
  '_score': 55.990067,
  '_source': {'location': 'Weißenberg, Saxony, Germany',
   'geoPoint': {'lon': 14.6604675, 'lat': 51.1977135}}},
 {'_index': 'coordinates_v2',
  '_type': 'coordinates',
  '_id': '55aa81e10cf2482db6507558',
  '_score': 52.889225,
  '_source': {'location': 'Weißenberg',
   'geoPoint': {'lon': 14.6604675, 'lat': 51.1977135}}},
 {'_index': 'coordinates_v2',
  '_type': 'coordinates',
  '_id': '554d6f6be4b00b455db905e3',
  '_score': 52.218285,
  '_source': {'location': 'Weißenberg, Deutschland',
   'geoPoint': {'lon': 14.6604675, 'lat': 51.1977135}}},
 {'_index': 'coordinates_v2',
  '_type': 'coordinates',
  '_id': '5659f5fd0cf25d2c0b7f6456',
  '_scor